# 03 - Star Formation History Analysis (Figure 2)

This notebook generates **Figure 2** from the paper using the exact original code, adapted for the refactored data structure.

**Goal**: Reproduce Figure 2 (Sample sSFHs) exactly as in the original paper.

In [ ]:
# Imports (adapted for refactored structure)
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import os
from scipy.integrate import simpson

# Add src to path
sys.path.append('../src')

from utils.analysis import (
    sim_name, colors, times,
    t_axis_label, sSFR_axis_label
)

%matplotlib inline

In [ ]:
# Load processed simulation data
with open('../data/sim_data_processed.pkl', 'rb') as f:
    sim_data = pickle.load(f)

print("Loaded processed simulation data:")
for sim in sim_name:
    if sim_data[sim] is not None:
        ngal = sim_data[sim]['ngal']
        print(f"{sim:>15s}: {ngal:>8,d} galaxies")
    else:
        print(f"{sim:>15s}: No data")

In [ ]:
# EXACT ORIGINAL CODE - Process normalized SFHs
# This follows the exact original normalization from the SFHs_figures.ipynb

# First normalize by integrated SFH (like original)
for sim in sim_name:
    if sim_data[sim] is None:
        continue
        
    ssfr = []
    sim_times = sim_data[sim]['times'][0] if sim_data[sim]['times'] is not None else times
    
    for i in range(len(sim_data[sim]['sm'])):
        # Use original normalization method exactly
        normalized_sfh = sim_data[sim]['sfh_raw'][i]/simpson(sim_data[sim]['sfh_raw'][i], x=sim_times*10**9)
        ssfr.append(normalized_sfh)
    
    sim_data[sim]['sfh'] = np.array(ssfr)
    
print("Normalized SFHs computed for all simulations")

In [ ]:
# Interpolate to common time grid (like original)
for sim in sim_name:
    if sim_data[sim] is None:
        continue
        
    # interpolate
    sim_times = sim_data[sim]['times'][0] if sim_data[sim]['times'] is not None else times
    sfh = sim_data[sim]['sfh']
    
    interp = []
    for i in range(sfh.shape[0]):
        fp = sfh[i]
        interp.append(np.interp(times, sim_times, fp)[1:137]) # cut to t = [0.1, 13.6] Gyr
    
    interp = np.array(interp)
    sim_data[sim]['sfh'] = interp
    
print("Interpolated SFHs to common time grid")

In [ ]:
# EXACT ORIGINAL CODE - Select sample SFHs by mass bins
# set seed for reproducability
np.random.seed(13)

# select randomly from mass bins
bin9 = {}
bin10 = {}
bin11 = {}
for sim in sim_name:
    if sim_data[sim] is None:
        bin9[sim] = []
        bin10[sim] = []
        bin11[sim] = []
        continue
        
    sm = sim_data[sim]['sm']
    sfh = sim_data[sim]['sfh']
    
    if (sim != 'Mufasa') & (sim != 'Simba'):
        if len(sm[(sm>1e9) & (sm<1e10)]) > 0:
            arg9 = np.random.randint(0,len(sm[(sm>1e9) & (sm<1e10)])-1,3)
        else:
            arg9 = [0, 0, 0]  # fallback
    
    if len(sm[(sm>1e10) & (sm<1e11)]) > 0:
        arg10 = np.random.randint(0,len(sm[(sm>1e10) & (sm<1e11)])-1,3)
    else:
        arg10 = [0, 0, 0]  # fallback
        
    if len(sm[(sm>1e11) & (sm<1e12)]) > 0:
        arg11 = np.random.randint(0,len(sm[(sm>1e11) & (sm<1e12)])-1,3)
    else:
        arg11 = [0, 0, 0]  # fallback
    
    bin9[sim] = []
    bin10[sim] = []
    bin11[sim] = []
    
    for i in range(3):
        if (sim != 'Mufasa') & (sim != 'Simba'):
            if len(sm[(sm>1e9) & (sm<1e10)]) > 0:
                bin9[sim].append(sfh[(sm>1e9) & (sm<1e10)][arg9[i]])
            else:
                bin9[sim].append(np.zeros_like(times[1:137]))  # Empty SFH
        else:
            bin9[sim].append(np.zeros_like(times[1:137]))  # Empty for Mufasa/Simba
            
        if len(sm[(sm>1e10) & (sm<1e11)]) > 0:
            bin10[sim].append(sfh[(sm>1e10) & (sm<1e11)][arg10[i]])
        else:
            bin10[sim].append(np.zeros_like(times[1:137]))
            
        if len(sm[(sm>1e11) & (sm<1e12)]) > 0:
            bin11[sim].append(sfh[(sm>1e11) & (sm<1e12)][arg11[i]])
        else:
            bin11[sim].append(np.zeros_like(times[1:137]))

print("Selected random SFHs from mass bins")

In [ ]:
# EXACT ORIGINAL PLOTTING CODE (from cell 14 in SFHs_figures.ipynb)
fig, axs = plt.subplots(7, 3, figsize=(12, 16))
plt.subplots_adjust(wspace=0, hspace=0)

for i,sim in enumerate(sim_name):
    for j in range(3):
        if j == 2:
            if (sim != 'Mufasa') & (sim != 'Simba'):
                axs[i,0].semilogy(times[1:137], bin9[sim][j], c=colors[sim], lw=2) 
            else:
                axs[i,0].semilogy(times[1:137], bin10[sim][j], c='w', lw=0) #just to get axis to scale
            axs[i,1].semilogy(times[1:137], bin10[sim][j], c=colors[sim], lw=2)
            axs[i,2].semilogy(times[1:137], bin11[sim][j], c=colors[sim], lw=2)
        else:
            if (sim != 'Mufasa') & (sim != 'Simba'):
                axs[i,0].semilogy(times[1:137], bin9[sim][j], c='grey', lw=1)
            else:
                axs[i,0].semilogy(times[1:137], bin10[sim][j], c='w', lw=0)
            axs[i,1].semilogy(times[1:137], bin10[sim][j], c='grey', lw=1)
            axs[i,2].semilogy(times[1:137], bin11[sim][j], c='grey', lw=1)
            
        # EXACT ORIGINAL tick formatting
        axs[i,j].tick_params(
            which='both',
            left=True, bottom=True,
            right= j==2,
            labelleft=(j == 0),        
            labelright=False,       
            labelbottom=(i == 6),
            labeltop=False,
            direction='in',
            labelsize=16
        )
        axs[i,j].set_ylim(10**-12.9,10**-9.1)
    
    # Y-axis labels for each row
    axs[i,0].set_ylabel(sSFR_axis_label, fontsize=20)
    
    # Simulation names on the right side
    if sim != 'UniverseMachine':
        axs[i,2].text(13.8,10**-9.7, sim, ha='right', fontsize=20, 
                      bbox=dict(facecolor='white', edgecolor='white', linewidth=0))
    else:
        axs[i,2].text(13.8,10**-9.7, 'UM', ha='right', fontsize=20, 
                      bbox=dict(facecolor='white', edgecolor='white', linewidth=0))

# Column titles
axs[0,0].set_title(r'$M_\star \in [10^9,10^{10}]\ \mathrm{M}_\odot$', fontsize=20)
axs[0,1].set_title(r'$M_\star \in [10^{10},10^{11}]\ \mathrm{M}_\odot$', fontsize=20)
axs[0,2].set_title(r'$M_\star \in [10^{11},10^{12}]\ \mathrm{M}_\odot$', fontsize=20)

# X-axis labels only on bottom row
for j in range(3):
    axs[6,j].set_xlabel(t_axis_label, fontsize=20)
        
plt.savefig('../figures/figure2_sample_sfhs.png', bbox_inches='tight', dpi=300)
plt.savefig('../figures/figure2_sample_sfhs.pdf', bbox_inches='tight')

plt.show()
plt.close()

## Summary

This notebook reproduces **Figure 2** exactly as in the original paper, showing sample specific star formation histories (sSFHs) for different mass bins.

Key features:
- 7x3 grid showing all simulations (rows) across 3 mass bins (columns)
- Mass bins: 10^9-10^10, 10^10-10^11, 10^11-10^12 M☉
- 3 random samples per mass bin, with the 3rd sample highlighted in color
- Mufasa/Simba excluded from lowest mass bin due to mass cuts
- Normalized specific star formation rates over cosmic time

**Files generated:**
- `../figures/figure2_sample_sfhs.png`
- `../figures/figure2_sample_sfhs.pdf`